# Cross-Selling

Se van a ver las posibles combinaciones más frecuentes de lo que pide la gente

In [5]:
import pandas as pd

## Se lee el archivo csv

In [6]:
df = pd.read_csv('data/sales_clean.csv')
df

,location,name,category,price,cost,utility,quantity,total,total_utility,Year,Month,Day_name,Day,Hour,sale_timestamp
0,University District,Iced Mocha,Coffee,5.19,2.71,2.48,2,10.38,4.96,2024,January,Tuesday,2,21,2024-01-02 21:52:00
1,Financial District,Sandwich,Food,2.83,1.48,1.35,1,2.83,1.35,2024,January,Thursday,18,21,2024-01-18 21:25:00
2,Suburbs,Cheesecake,Bakery,2.82,1.47,1.35,1,2.82,1.35,2024,March,Monday,18,5,2024-03-18 05:13:00
3,University District,Cheesecake,Bakery,2.82,1.47,1.35,1,2.82,1.35,2024,January,Monday,1,13,2024-01-01 13:57:00
4,Suburbs,Smoothie,Food,3.94,2.06,1.88,1,3.94,1.88,2024,March,Wednesday,6,6,2024-03-06 06:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,Downtown,Salad,Food,3.37,1.76,1.61,1,3.37,1.61,2024,January,Monday,8,10,2024-01-08 10:38:00
14996,Suburbs,Americano,Coffee,4.69,2.45,2.24,3,14.07,6.72,2024,January,Wednesday,3,22,2024-01-03 22:14:00
14997,Mall,Wrap,Food,4.36,2.28,2.08,2,8.72,4.16,2024,March,Sunday,3,21,2024-03-03 21:57:00
14998,Financial District,Wrap,Food,4.36,2.28,2.08,2,8.72,4.16,2024,March,Friday,22,9,2024-03-22 09:09:00


## Convertimos a tipo de dato 'fecha'

In [7]:
df['sale_timestamp'] = pd.to_datetime(df['sale_timestamp'])
df['sale_timestamp'].dtype

dtype('<M8[ns]')

## Se inicia con el proceso de Cross-Selling

In [9]:
# Se crea una columna de fecha redondeado hasta minutos
df['sale_date'] = df['sale_timestamp'].dt.date

In [10]:
# Se crea un índice unico, que ayuda a agrupar productos
df['transaction_id'] = df['location'] + '_' + df['sale_date'].astype(str) + '_' + df['Hour'].astype(str)

# Se crea una tabla pivot agrupando los productos por su índice, para saber cuantos han vendido en esa fecha y lugar exacto
basket = df.groupby(['transaction_id', 'name'])['quantity'].sum().unstack().fillna(0)

# Se pasa la tabla en código binario, porque el algoritmo solo trabaja en código binario
basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

/var/folders/1w/_fkk8d6978j_747fm_9qxcww0000gn/T/ipykernel_1561/1854172550.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)


In [13]:
# Se importan las funciones de la libreria
import warnings
from mlxtend.frequent_patterns import apriori, association_rules

# Ignoramos los warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Buscar combinaciones con al menos 1% de soporte
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True, verbose=1)

# Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Ver reglas ordenadas por confianza
rules.sort_values(by="confidence", ascending=False).head()

Processing 380 combinations | Sampling itemset size 2


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
